In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dropout
from keras.optimizers import SGD
from math import sqrt
from matplotlib import pyplot
import matplotlib.dates as mdates
from numpy import array
import random

C:\Users\PC\AppData\Local\Temp\ipykernel_21396\267029996.py:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [2]:
random.seed(42)

In [3]:
 # définition d'un parser pour le chargement des jeux de données sous forme de sériex temporellex
def parser(x):
    return datetime.strptime(x,'%d/%m/%Y')

In [ ]:
#Chargement des données de ventes
top_data = pd.read_csv('data_mois.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
lens_data = pd.read_csv('Lens_full.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
lens_care_data = pd.read_csv('Lens_care_full.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
lens_cus_data = pd.read_csv('Lens_clients.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
lenscare_cus_data =  pd.read_csv('Lens_care_clients.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

#Chargement des prévisions de l'entreprise 
plan = pd.read_csv('Plan_full.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
lens_plan = pd.read_csv('Plan_lens.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
lens_care_plan = pd.read_csv('Plan_LC.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
lens_cus_plan = pd.read_csv('Plan_client_lens.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
lens_carecus_plan = pd.read_csv('Plan_client_LC.csv', sep=";", parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

                            

In [4]:
# Conversion des séries temporelles en problème de machine learning supervisé
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# Différenciation des séries temporelles
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)
 
# Préparation des données
def prepare_data(series, n_test, n_lag, n_seq):
 # extraction des valeurs
    raw_values = series.values
 # stationnarisation
    diff_series = difference(raw_values, 1)
    diff_values = diff_series.values
    diff_values = diff_values.reshape(len(diff_values), 1)
 # normalisation
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(diff_values)  
    scaled_values = scaled_values.reshape(len(scaled_values), 1)
 # mise en forme (problème de machine learning supervisé)
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values
 # division du jeu de données : jeu d'entrainement et jeu de test
    train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
    return scaler, train, test
 
# Création du réseau de neurones LSTM à 5 couches
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4):
 # Mise en forme des données : tableau 3D
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
 # Structure du réseau
    model = Sequential()
    model.add(LSTM(n_neurons, return_sequences=True, input_shape= (X.shape[1], 1)))
    model.add(Dropout(0.5))
    model.add(LSTM(n_neurons_2, return_sequences=True, input_shape= (X.shape[1], 1)))
    model.add(Dropout(0.5))
    model.add(LSTM(n_neurons_3, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(n_neurons_4))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
 # Apprentissage
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model
 
# Prévision 
def forecast_lstm(model, X, n_batch):
 # Mise en forme : tableau 3D
    X = X.reshape(1, 1, len(X))
 # Prévision
    forecast = model.predict(X, batch_size=n_batch)
    return [x for x in forecast[0, :]]
 
# Prévision
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
 # Prévision
        forecast = forecast_lstm(model, X, n_batch)
 # Stockage des prévisions
        forecasts.append(forecast)
    return forecasts
 
# Inversion de la différenciation
def inverse_difference(last_ob, forecast):
    inverted = list()
    inverted.append(forecast[0] + last_ob)
    for i in range(1, len(forecast)):
        inverted.append(forecast[i] + inverted[i-1])
    return inverted
 
# Traitement inverse (pour pouvoir comparer les prévisions avec les données réelles) 
def inverse_transform(series, forecasts, scaler, n_test):
    inverted = list()
    for i in range(len(forecasts)):
        forecast = array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
 # Inversion de la normalisation
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]
 # Inversion de la differenciation
        index = len(series) - n_test + i - 1
        last_ob = series.values[index]
        inv_diff = inverse_difference(last_ob, inv_scale)
        inverted.append(inv_diff)
    return inverted
 
# Evaluation des prévisions
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
        mape = mean_absolute_percentage_error(actual, predicted)
        mse = mean_squared_error(actual, predicted)
        rmse = sqrt(mean_squared_error(actual, predicted))
        print('t+%d MSE: %f' % ((i+1), mse))
        print('t+%d RMSE: %f' % ((i+1), rmse))
        print('t+%d MAPE: %f' % ((i+1), mape))
        
        
# Création d'un dataframe contenant les métriques obtenues à partir des prévisions
def get_forecast_metrics(df_metrics, col, test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
        mape = mean_absolute_percentage_error(actual, predicted)
        mse = mean_squared_error(actual, predicted)
        rmse = sqrt(mean_squared_error(actual, predicted))
        ar = np.array([mape,mse,rmse])
        name = col + str(i+1)
        df_metrics[name] = ar
    return df_metrics

#Stockage des prévisions dans un dataframe
def get_forecast(df_forecasts, col, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        predicted = [forecast[i] for forecast in forecasts]
        name = col + str(i+1)
        df_forecasts[name] = predicted
    return df_forecasts

# Affichage graphique des ventes réelles et des prévisions
def plot_forecasts(series, forecasts, n_test):
 # Affichage des ventes réelles
    pyplot.plot(series.values)
 # Affichage des prévisions
    for i in range(len(forecasts)):
        off_s = len(series) - n_test + i - 1
        off_e = off_s + len(forecasts[i]) + 1
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = [series.values[off_s]] + forecasts[i]
        pyplot.plot(xaxis, yaxis, color='orange')
        
    pyplot.ticklabel_format(style='plain', axis='y')
 # Affichage du graphique
    pyplot.show()

In [5]:
def fit_GRU(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4):
 # Mise en forme : tableau 3D
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    
# Structure du réseau GRU
    regressorGRU = Sequential()
    # Première couche GRU
    regressorGRU.add(GRU(units=n_neurons, return_sequences=True, input_shape=(X.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.5))
    # Deuxième couche GRU
    regressorGRU.add(GRU(units=n_neurons_2, return_sequences=True, input_shape=(X.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.5))
    # Troisième couche GRU
    regressorGRU.add(GRU(units=n_neurons_3, return_sequences=True, input_shape=(X.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.5))
    # Quatrième couche GRU
    regressorGRU.add(GRU(units=n_neurons_4, activation='tanh'))
    regressorGRU.add(Dropout(0.5))
    # Couche de sortie
    regressorGRU.add(Dense(units=1))
    regressorGRU.compile(optimizer='adam', loss='mean_squared_error')
 # Entrainement du modèle 
    for i in range(nb_epoch):
        regressorGRU.fit(X, y, epochs=1, batch_size=n_batch)
        regressorGRU.reset_states()
    return regressorGRU
 
# Prévisions à l'aide d'un GRU
def forecast_GRU(model, X, n_batch):
 # Mise en forme des données
    X = X.reshape(1, 1, len(X))
 # prévisions
    forecast = model.predict(X, batch_size=n_batch)
 # Conversion en array
    return [x for x in forecast[0, :]]

def make_forecasts_GRU(model, n_batch, train, test, n_lag, n_seq):
    forecasts = []
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        
        # Faire les prévisions
        forecast = []
        for j in range(n_seq):
            forecast.append(forecast_GRU(model, X, n_batch))
            X = np.concatenate((X[1:], forecast[-1]), axis=0)
        
        # Enregistrer les prévisions
        forecasts.append(forecast)
    
    return forecasts

In [6]:
def plan_vs_forecast(plan, prevision, actual):
    #calcul des métriques d'erreur
    mape_p = mean_absolute_percentage_error(plan, actual)
    mse_p = mean_squared_error(plan, actual)
    rmse_p = sqrt(mean_squared_error(plan, actual))
    mape = mean_absolute_percentage_error(prevision, actual)
    mse = mean_squared_error(prevision, actual)
    rmse = sqrt(mean_squared_error(prevision, actual))

    dates = pd.to_datetime(['2023-01-01', '2023-02-01', '2023-03-01'])
    #Prévision de l'entreprise
    y1 = plan.values              

    #Prévision du modèle
    y2 = prevision              
    
    #Valeurs réelles
    y3 = actual         

# Tracer les courbes
    pyplot.plot(dates, y1, label='Plan')
    pyplot.plot(dates, y2, label='Forecast')
    pyplot.plot(dates, y3, label='Actual')


# Ajouter une légende et des titres
    pyplot.legend()
    pyplot.title('Comparaison de deux courbes')
    pyplot.xlabel('Dates')
    pyplot.ylabel('Sales')
    
    ax = pyplot.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    
    
    pyplot.ticklabel_format(style='plain', axis='y')

# Afficher le graphique
    pyplot.show()
#Afficher les métriques    
    print('mape plan : ' + str(mape_p))
    print('mape forecast : ' + str(mape))
    print('mse plan : ' + str(mse_p))
    print('mse forecast : ' + str(mse))
    print('rmse plan : ' + str(rmse_p))
    print('rmse forecast : ' + str(rmse))

    #Calcul des métriques moyennes
def avg_metrics(plan, previsions, actual):
    mape_p = 0
    mape = 0
    mse_p = 0
    mse = 0
    rmse_p = 0
    rmse = 0
    cols = list(actual.columns)
    for col in cols:
        mape_p = mape_p + mean_absolute_percentage_error(plan[col], actual[col])
        mse_p = mse_p + mean_squared_error(plan[col], actual[col])
        rmse_p = rmse_p + sqrt(mean_squared_error(plan[col], actual[col]))
        mape = mape + mean_absolute_percentage_error(previsions[col], actual[col])
        mse = mse + mean_squared_error(previsions[col], actual[col])
        rmse = rmse + sqrt(mean_squared_error(previsions[col], actual[col]))

    avg_mape_p = mape_p/len(cols)
    avg_mape = mape/len(cols)
    avg_mse_p = mse_p/len(cols)
    avg_mse = mse/len(cols)
    avg_rmse_p = rmse_p/len(cols)
    avg_rmse = rmse/len(cols)
    print('mape moyenne plan : ' + str(avg_mape_p))
    print('mape moyenne forecast : ' + str(avg_mape))
    print('mse moyenne plan : ' + str(avg_mse_p))
    print('mse moyenne forecast : ' + str(avg_mse))
    print('rmse moyenne plan : ' + str(avg_rmse_p))
    print('rmse moyenne forecast : ' + str(avg_rmse))

In [7]:
#Affichage d'un graphique de comparaison du plan et des prévisions des réseaux de neurones
def plan_vs_forecast_2(plan, prevision_LSTM, previson_GRU, actual, titre):
    #calcul des métriques d'erreur
    mape_p = mean_absolute_percentage_error(plan, actual)
    mse_p = mean_squared_error(plan, actual)
    rmse_p = sqrt(mean_squared_error(plan, actual))
    mape_LSTM = mean_absolute_percentage_error(prevision_LSTM, actual)
    mse_LSTM = mean_squared_error(prevision_LSTM, actual)
    rmse_LSTM = sqrt(mean_squared_error(prevision_LSTM, actual))
    mape_GRU = mean_absolute_percentage_error(previson_GRU, actual)
    mse_GRU = mean_squared_error(previson_GRU, actual)
    rmse_GRU = sqrt(mean_squared_error(previson_GRU, actual))

    dates = pd.to_datetime(['2023-01-01', '2023-02-01', '2023-03-01'])
    #Prévision de l'entreprise
    y1 = plan.values              

    #Prévision du modèle LSTM
    y2 = prevision_LSTM              
    
    #Prévision du modèle GRU
    y3 = previson_GRU                
    
    #Valeurs réelles
    y4 = actual         

# Tracer les courbes
    pyplot.plot(dates, y1, label='Plan')
    pyplot.plot(dates, y2, label='Prévisions LSTM')
    pyplot.plot(dates, y3, label='Prévisions GRU')
    pyplot.plot(dates, y4, label='Ventes réelles')


# Ajouter une légende et des titres
    pyplot.legend()
    pyplot.title(titre)
    pyplot.xlabel('Dates')
    pyplot.ylabel('Ventes')
    
    ax = pyplot.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    
    
    pyplot.ticklabel_format(style='plain', axis='y')

# Afficher le graphique
    pyplot.show()
#Afficher les métriques    
    print('mape plan : ' + str(mape_p))
    print('mape LSTM : ' + str(mape_LSTM))
    print('mape GRU : ' + str(mape_GRU))
    print('mse plan : ' + str(mse_p))
    print('mse LSTM : ' + str(mse_LSTM))
    print('mse GRU : ' + str(mse_GRU))    
    print('rmse plan : ' + str(rmse_p))
    print('rmse LSTM : ' + str(rmse_LSTM))
    print('rmse GRU : ' + str(rmse_GRU))    

### I. Top data

#### 1.1 Forecast

#### 1.1.1 LSTM

In [ ]:
series = top_data

In [ ]:
# configuration
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 1500
n_batch = 10
n_neurons = 128
n_neurons_2 = 64
n_neurons_3 = 64
n_neurons_4 = 25
df_metrics = pd.DataFrame()
# Préparation des données
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# Prévisions à partir du modèle
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse des données
forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test+2)

# évaluation des prévisions
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# Affichage des prévisions
plot_forecasts(series, forecasts, n_test+2)

#Stockage des prévisions pour le premier trimestre 2023
lstm_forecasts_t_Q1 = forecasts[2]

#### 1.1.2 GRU

In [ ]:
#Configuration
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 1500
n_batch = 10
n_neurons = 200
n_neurons_2 = 200
n_neurons_3 = 150
n_neurons_4 = 100
# Préparation des données
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
model = fit_GRU(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# Prévisions à partir du modèle
forecasts = make_forecasts_GRU(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse des données
forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test+2)
# évaluation des prévisions
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# Affichage des prévisions
plot_forecasts(series, forecasts, n_test+2)

#Stockage des prévisions pour le premier trimestre 2023
GRU_forecasts_t_Q1 = forecasts[0]

#### 1.2 Comparaison avec le plan 

In [ ]:
plan_t_Q1 = plan[0:3]
actual_t_Q1 = actual[2]

In [ ]:
plan_vs_forecast(plan_t_Q1, lstm_forecasts_t_Q1, actual_t_Q1)

In [ ]:
plan_vs_forecast(plan_t_Q1, GRU_forecasts_t_Q1, actual_t_Q1)

In [ ]:
plan_vs_forecast_2(plan_t_Q1,lstm_forecasts_t_Q1, GRU_forecasts_t_Q1, actual_t_Q1, "Prévisions du total des ventes")

## II. Lens / Lens care data

### 2.1 Lens

#### 2.1.1 Forecasts

#### 2.1.1.1 LSTM

In [ ]:
series = lens_data

In [ ]:
# configuration
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 1500
n_batch = 10
n_neurons = 128
n_neurons_2 = 64
n_neurons_3 = 64
n_neurons_4 = 25
# Préparation des données
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# Prévisions à partir du modèle
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse des données
forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test+2)

# évaluation des prévisions
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# Affichage des prévisions
plot_forecasts(series, forecasts, n_test+2)

#Stockage des previsions pour le premier semestre 2023
lstm_forecasts_l_Q1 = forecasts[2]

#### 2.1.1.1 GRU

In [ ]:
#Configuration
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 1500
n_batch = 10
n_neurons = 200
n_neurons_2 = 200
n_neurons_3 = 150
n_neurons_4 = 100
# Préparation des données
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# fit model
model = fit_GRU(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# make forecasts
forecasts = make_forecasts_GRU(model, n_batch, train, test, n_lag, n_seq)
# inverse transform forecasts and test
forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test+2)
#Nom du client
# evaluate forecasts
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# plot forecasts
plot_forecasts(series, forecasts, n_test+2)
GRU_forecasts_l_Q1 = forecasts[0]

#### 2.1.2 Comparaison avec le plan

In [ ]:
lens_plan_Q1 = lens_plan[0:3]
actual_l_Q1 = actual[2]

In [ ]:
plan_vs_forecast(lens_plan_Q1, lstm_forecasts_l_Q1, actual_l_Q1)

In [ ]:
plan_vs_forecast(lens_plan_Q1, GRU_forecasts_l_Q1, actual_l_Q1)

In [ ]:
plan_vs_forecast_2(lens_plan_Q1,lstm_forecasts_l_Q1, GRU_forecasts_l_Q1, actual_l_Q1, "Prévisions des ventes de lentilles")

### 2.2 Lens care

#### 2.2.1 Forecast

#### 2.2.1.1 LSTM

In [ ]:
series = lens_care_data

In [ ]:
# configure
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 1500
n_batch = 10
n_neurons = 128
n_neurons_2 = 64
n_neurons_3 = 64
n_neurons_4 = 25
# Préparation des données
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# Prévisions à partir du modèle
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse des données
forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test+2)

# évaluation des prévisions
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# Affichage des prévisions
plot_forecasts(series, forecasts, n_test+2)


lstm_forecasts_lc_Q1 = forecasts[2]

#### 2.1.1.1 GRU

In [ ]:
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 1500
n_batch = 10
n_neurons = 200
n_neurons_2 = 200
n_neurons_3 = 150
n_neurons_4 = 100
# Préparation des données
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
model = fit_GRU(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
#  Prévisions à partir du modèle
forecasts = make_forecasts_GRU(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse des données
forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test+2)
# évaluation des prévisions
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# Affichage des prévisions
plot_forecasts(series, forecasts, n_test+2)
GRU_forecasts_lc_Q1 = forecasts[2]

#### 2.2.2 Comparaison avec le plan

In [ ]:
lens_care_plan_Q1 = lens_care_plan[0:3]
actual_lc_Q1 = actual[2]

In [ ]:
plan_vs_forecast(lens_care_plan_Q1, lstm_forecasts_lc_Q1, actual_lc_Q1)

In [ ]:
plan_vs_forecast(lens_care_plan_Q1, GRU_forecasts_lc_Q1, actual_lc_Q1)

In [ ]:
plan_vs_forecast_2(lens_care_plan_Q1,lstm_forecasts_lc_Q1, GRU_forecasts_lc_Q1, actual_lc_Q1, "Prévisions des ventes de lentilles")

## 3. Clients

### 3.1 Clients Lens

#### 3.1.1 Forecast

#### 3.1.1.1 LSTM

In [ ]:
data = lens_cus_data

In [ ]:
# configuration
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 500
n_batch = 10
n_neurons = 128
n_neurons_2 = 64
n_neurons_3 = 64
n_neurons_4 = 25

df_metrics = pd.DataFrame()
df_forecasts = pd.DataFrame()

for col in data:
    series = data[col]
# Préparation des données
    scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
    model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# Prévisions
    forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse
    forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
    actual = [row[n_lag:] for row in test]
    actual = inverse_transform(series, actual, scaler, n_test+2)
# Affichage du nom de la catégorie de client
    print(col)
# Evaluation des prévisions
    evaluate_forecasts(actual, forecasts, n_lag, n_seq)
    df_metrics = get_forecast_metrics(df_metrics,col, actual, forecasts, n_lag, n_seq)
    get_forecast(df_forecasts, col, forecasts, n_lag, n_seq)
# Affichage des prévisions
    plot_forecasts(series, forecasts, n_test+2)
#plan vs forecast (Q1)
    #print(forecasts)
    #actual_Q1 = actual[2]
    #plan_Q1 = data_plan[col][0:3]
    #forecasts_lstm_Q1 = forecasts[2]
    #plan_vs_forecast(plan_Q1, lstm_forecasts_Q1, actual_Q1)

In [ ]:
df_metrics

In [ ]:
df_metrics.to_csv("Metrics clients lens.csv")

In [ ]:
df_forecasts

In [ ]:
df_forecasts_Q1 = df_forecasts[2:3]

In [ ]:
clients = list(lens_cus_data.columns)
forecast_cus_l_Q1 = pd.DataFrame(columns=clients)
forecast_cus_l_Q1

In [ ]:
clients = list(lens_cus_data.columns)
forecast_cus_l_Q1 = pd.DataFrame(columns=clients)
for client in clients:
    x = []
    for i in range(1,4):
        col_name = client+str(i)
        y = df_forecasts_Q1[col_name].iloc[0]
        x.append(y)
    forecast_cus_l_Q1[client] = x

forecast_cus_l_Q1 

In [ ]:
forecast_cus_l_Q1[""]=['2023-01-01', '2023-02-01', '2023-03-01']
forecast_cus_l_Q1.set_index([""], inplace=True)
forecast_cus_l_Q1

In [ ]:
lens_cus_data

In [ ]:
actual_lens_cus_Q1 = lens_cus_data[60:63]
actual_lens_cus_Q1

In [ ]:
lens_cus_plan_Q1 = lens_cus_plan[0:3]
lens_cus_plan_Q1

#### 3.1.1.2 GRU

In [ ]:
# configuration
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 500
n_batch = 10
n_neurons = 200
n_neurons_2 = 200
n_neurons_3 = 150
n_neurons_4 = 100

df_metrics_l_GRU = pd.DataFrame()
df_forecasts_l_GRU = pd.DataFrame()

for col in data:
    series = data[col]
# Préparation des données
    scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
    model = fit_GRU(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# Prévisions
    forecasts = make_forecasts_GRU(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse
    forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
    actual = [row[n_lag:] for row in test]
    actual = inverse_transform(series, actual, scaler, n_test+2)
# Affichage du nom du client
    print(col)
# Evaluation des prévisions
    evaluate_forecasts(actual, forecasts, n_lag, n_seq)
    df_metrics = get_forecast_metrics(df_metrics_l_GRU,col, actual, forecasts, n_lag, n_seq)
    get_forecast(df_forecasts_l_GRU, col, forecasts, n_lag, n_seq)
# Affichage des prévisions
    plot_forecasts(series, forecasts, n_test+2)
#plan vs forecast (Q1)
    #print(forecasts)
    #actual_Q1 = actual[2]
    #plan_Q1 = data_plan[col][0:3]
    #forecasts_lstm_Q1 = forecasts[2]
    #plan_vs_forecast(plan_Q1, lstm_forecasts_Q1, actual_Q1)

In [ ]:
df_metrics_l_GRU

In [ ]:
df_metrics_l_GRU.to_csv("Metrics clients lens GRU.csv")

In [ ]:
df_forecasts_l_GRU_Q1 = df_forecasts_l_GRU[2:3]

In [ ]:
clients = list(lens_cus_data.columns)
forecast_cus_l_GRU_Q1 = pd.DataFrame(columns=clients)
for client in clients:
    x = []
    for i in range(1,4):
        col_name = client+str(i)
        y = df_forecasts_l_GRU_Q1[col_name].iloc[0]
        x.append(y)
    forecast_cus_l_GRU_Q1[client] = x

forecast_cus_l_GRU_Q1 

In [ ]:
forecast_cus_l_GRU_Q1[""]=['2023-01-01', '2023-02-01', '2023-03-01']
forecast_cus_l_GRU_Q1.set_index([""], inplace=True)
forecast_cus_l_GRU_Q1

In [ ]:
#Comparaison avec plan

In [ ]:
for col in list(lens_cus_data.columns):
    print(col)
    plan_vs_forecast(lens_cus_plan_Q1[col], forecast_cus_l_Q1[col], actual_lens_cus_Q1[col])

In [ ]:
for col in list(lens_cus_data.columns):
    print(col)
    plan_vs_forecast(lens_cus_plan_Q1[col], forecast_cus_l_GRU_Q1[col], actual_lens_cus_Q1[col])

In [ ]:
for col in list(lens_cus_data.columns):
    print(col)
    plan_vs_forecast_2(lens_cus_plan_Q1[col],forecast_cus_l_Q1[col], forecast_cus_l_GRU_Q1[col], actual_lens_cus_Q1[col], "Prévisions des ventes de lentilles par catégorie de clients")

In [ ]:
avg_metrics(lens_cus_plan_Q1, forecast_cus_l_Q1, actual_lens_cus_Q1)

In [ ]:
avg_metrics(lens_cus_plan_Q1, forecast_cus_l_GRU_Q1, actual_lens_cus_Q1)

### 3.2 Lens care

#### 3.2.1 Forecast

#### 3.2.1.1 LSTM

In [ ]:
data = lenscare_cus_data

In [ ]:
# configuration
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 500
n_batch = 10
n_neurons = 128
n_neurons_2 = 64
n_neurons_3 = 64
n_neurons_4 = 25

df_metrics_lc_LSTM = pd.DataFrame()
df_forecasts_lc_LSTM = pd.DataFrame()

for col in data:
    series = data[col]
# Préparation des données
    scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
    model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# Prévision
    forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse des prévisions
    forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
    actual = [row[n_lag:] for row in test]
    actual = inverse_transform(series, actual, scaler, n_test+2)
# Affichage du nom de la catégorie de client
    print(col)
# Evaluation des prévisions
    evaluate_forecasts(actual, forecasts, n_lag, n_seq)
    df_metrics_lc_LSTM = get_forecast_metrics(df_metrics_lc_LSTM, col, actual, forecasts, n_lag, n_seq)
    get_forecast(df_forecasts_lc_LSTM, col, forecasts, n_lag, n_seq)
# Affichage des prévisions
    plot_forecasts(series, forecasts, n_test+2)
#plan vs forecast (Q1)
    #print(forecasts)
    #actual_Q1 = actual[2]
    #plan_Q1 = data_plan[col][0:3]
    #forecasts_lstm_Q1 = forecasts[2]
    #plan_vs_forecast(plan_Q1, lstm_forecasts_Q1, actual_Q1)

In [ ]:
df_metrics_lc_LSTM

In [ ]:
df_metrics_lc_LSTM.to_csv("Metrics clients lens care LSTM.csv")

In [ ]:
df_forecasts_lc_LSTM

In [ ]:
df_forecasts_lc_LSTM_Q1 = df_forecasts_lc_LSTM[2:3]

In [ ]:
clients = list(lenscare_cus_data.columns)
forecast_cus_lc_Q1_LSTM = pd.DataFrame(columns=clients)
for client in clients:
    x = []
    for i in range(1,4):
        col_name = client+str(i)
        y = df_forecasts_lc_LSTM[col_name].iloc[0]
        x.append(y)
    forecast_cus_lc_Q1_LSTM[client] = x

forecast_cus_lc_Q1_LSTM 

In [ ]:
forecast_cus_lc_Q1_LSTM[""]=['2023-01-01', '2023-02-01', '2023-03-01']
forecast_cus_lc_Q1_LSTM.set_index([""], inplace=True)
forecast_cus_lc_Q1_LSTM

In [ ]:
lenscare_cus_data

In [ ]:
lenscare_cus_data

In [ ]:
actual_lenscare_cus_Q1 = lenscare_cus_data[60:63]
actual_lenscare_cus_Q1

In [ ]:
lens_carecus_plan_Q1 = lens_carecus_plan[0:3]
lens_carecus_plan_Q1

In [ ]:
# configuration
n_lag = 1
n_seq = 3
n_test = 4
n_epochs = 500
n_batch = 10
n_neurons = 200
n_neurons_2 = 200
n_neurons_3 = 150
n_neurons_4 = 100

df_metrics_lc_GRU = pd.DataFrame()
df_forecasts_lc_GRU = pd.DataFrame()

for col in data:
    series = data[col]
# Préparation des données
    scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
# Entrainement du modèle
    model = fit_GRU(train, n_lag, n_seq, n_batch, n_epochs, n_neurons, n_neurons_2, n_neurons_3, n_neurons_4)
# Prévision
    forecasts = make_forecasts_GRU(model, n_batch, train, test, n_lag, n_seq)
# Transformation inverse
    forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
    actual = [row[n_lag:] for row in test]
    actual = inverse_transform(series, actual, scaler, n_test+2)
# Affichage du nom de la catégorie de client
    print(col)
# Evaluation des prévisions
    evaluate_forecasts(actual, forecasts, n_lag, n_seq)
    df_metrics_lc_GRU = get_forecast_metrics(df_metrics_lc_GRU,col, actual, forecasts, n_lag, n_seq)
    get_forecast(df_forecasts_lc_GRU, col, forecasts, n_lag, n_seq)
# Affichage des prévisions
    plot_forecasts(series, forecasts, n_test+2)
#plan vs forecast (Q1)
    #print(forecasts)
    #actual_Q1 = actual[2]
    #plan_Q1 = data_plan[col][0:3]
    #forecasts_lstm_Q1 = forecasts[2]
    #plan_vs_forecast(plan_Q1, lstm_forecasts_Q1, actual_Q1)

In [ ]:
df_metrics_lc_GRU

In [ ]:
df_metrics_lc_GRU.to_csv("Metrics clients lens care GRU.csv")

In [ ]:
df_forecasts_lc_GRU_Q1 = df_forecasts_lc_GRU[2:3]

In [ ]:
clients = list(lenscare_cus_data.columns)
forecast_cus_lc_GRU_Q1 = pd.DataFrame(columns=clients)
for client in clients:
    x = []
    for i in range(1,4):
        col_name = client+str(i)
        y = df_forecasts_lc_GRU_Q1[col_name].iloc[0]
        x.append(y)
    forecast_cus_lc_GRU_Q1[client] = x

forecast_cus_lc_GRU_Q1 

In [ ]:
forecast_cus_lc_GRU_Q1[""]=['2023-01-01', '2023-02-01', '2023-03-01']
forecast_cus_lc_GRU_Q1.set_index([""], inplace=True)
forecast_cus_lc_GRU_Q1

In [ ]:
# Comparaison avec le plan

In [ ]:
for col in list(lenscare_cus_data.columns):
    print(col)
    plan_vs_forecast(lens_carecus_plan_Q1[col], forecast_cus_lc_Q1_LSTM[col], actual_lenscare_cus_Q1[col])

In [ ]:
for col in list(lenscare_cus_data.columns):
    print(col)
    plan_vs_forecast(lens_carecus_plan_Q1[col], forecast_cus_lc_GRU_Q1[col], actual_lenscare_cus_Q1[col])

In [ ]:
for col in list(lenscare_cus_data.columns):
    print(col)
    plan_vs_forecast_2(lens_carecus_plan_Q1[col],forecast_cus_lc_Q1_LSTM[col], forecast_cus_lc_GRU_Q1[col], actual_lenscare_cus_Q1[col], "Prévisions des ventes de lentilles par catégorie de clients")

In [ ]:
forecast_cus_lc_Q1_LSTM

In [ ]:
forecast_cus_lc_GRU_Q1

In [ ]:
avg_metrics(lens_carecus_plan_Q1, forecast_cus_lc_Q1_LSTM, actual_lenscare_cus_Q1)

In [ ]:
avg_metrics(lens_carecus_plan_Q1, forecast_cus_lc_GRU_Q1, actual_lenscare_cus_Q1)

In [ ]:
#Bottom up 

In [ ]:
l_LSTM = forecast_cus_l_Q1.sum(axis=1)
l_GRU = forecast_cus_l_GRU_Q1.sum(axis=1)
lc_LSTM = forecast_cus_lc_Q1_LSTM.sum(axis=1)
lc_GRU = forecast_cus_lc_GRU_Q1.sum(axis=1)

In [ ]:
t_LSTM = l_LSTM + lc_LSTM
t_GRU = l_GRU + lc_GRU

In [ ]:
print(l_LSTM)
print(l_GRU)
print(lc_LSTM)
print(lc_GRU)

In [ ]:
#Comparaison avec le plan après bottom up

In [ ]:
plan_vs_forecast_2(lens_care_plan_Q1,lc_LSTM, lc_GRU, actual_lc_Q1, "Prévisions des ventes de lentilles")

In [ ]:
plan_vs_forecast_2(lens_plan_Q1,l_LSTM, l_GRU, actual_l_Q1, "Prévisions des ventes de lentilles")

In [ ]:
plan_vs_forecast_2(plan_t_Q1,t_LSTM, t_GRU, actual_t_Q1, "Prévisions du total des ventes")